In [1]:
# import pandas as pd

# df = pd.read_csv('/kaggle/input/data-cleaned/preprocessed_cv_text_data.csv')
# df.head()

,Category,Text,Processed_Text
0,-net-developer,Contact [email protected] +61 412 345 678 Skil...,61 412 345 678 skill c net framework aspnet sq...
1,-net-developer,Contact [email protected] +55 (11) 98765-4321 ...,55 11 987654321 skill c net framework aspnet c...
2,-net-developer,Contact [email protected] +39 02 1234 5678 Ski...,39 02 1234 5678 skill c net framework aspnet s...
3,-net-web-developer,Contact [email protected] +55 11 91234-5678 Sk...,55 11 912345678 skill aspnet c mvc agile metho...
4,-net-web-developer,Contact [email protected] +81 90-1234-5678 Ski...,81 9012345678 skill c net aspnet mvc sql serve...


## RAG

In [38]:
import pandas as pd
import re
# file_path = "/kaggle/input/data-preprocess/combined_text.csv"
def remove_first_number_group(text):
    # Match either:
    # 1. A group of 2–5 space-separated digit blocks
    # 2. OR a single long number (8+ digits)
    match = re.search(r'((\d+\s+){1,4}\d+|\d{8,})', text)
    if match:
        return text[:match.start()].rstrip() + ' ' + text[match.end():].lstrip()
    return text

file_path = '/kaggle/input/data-cleaned/preprocessed_cv_text_data.csv'
df = pd.read_csv(file_path)
df['Combined_Text'] = df['Category'] + " " + df['Processed_Text']
df['Combined_Text'] = df['Combined_Text'].str.replace('-', ' ', regex=True)
df['Combined_Text'] = df['Combined_Text'].str.replace(r'\s+', ' ', regex=True).str.strip()
df['Combined_Text'] = df['Combined_Text'].apply(remove_first_number_group)
df.head()

,Category,Text,Processed_Text,Combined_Text
0,-net-developer,Contact [email protected] +61 412 345 678 Skil...,61 412 345 678 skill c net framework aspnet sq...,net developer skill c net framework aspnet sql...
1,-net-developer,Contact [email protected] +55 (11) 98765-4321 ...,55 11 987654321 skill c net framework aspnet c...,net developer skill c net framework aspnet cor...
2,-net-developer,Contact [email protected] +39 02 1234 5678 Ski...,39 02 1234 5678 skill c net framework aspnet s...,net developer skill c net framework aspnet sql...
3,-net-web-developer,Contact [email protected] +55 11 91234-5678 Sk...,55 11 912345678 skill aspnet c mvc agile metho...,net web developer skill aspnet c mvc agile met...
4,-net-web-developer,Contact [email protected] +81 90-1234-5678 Ski...,81 9012345678 skill c net aspnet mvc sql serve...,net web developer skill c net aspnet mvc sql s...


In [39]:
def chunk_text(text, max_length=1000, overlap=200):

    chunks = []
    start = 0
    while start < len(text):
        end = start + max_length
        chunks.append(text[start:end])
        start += max_length - overlap  
    return chunks

all_chunks = []

for idx, row in df.iterrows():
    chunks = chunk_text(row['Combined_Text'])
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            'cv_id': idx,
            'chunk_id': i,
            'text': chunk
        })

# Tạo DataFrame chứa các chunk
chunk_df = pd.DataFrame(all_chunks)
chunk_df.head()

,cv_id,chunk_id,text
0,0,0,net developer skill c net framework aspnet sql...
1,0,1,017 feb 2021 contributed development ecommerce...
2,1,0,net developer skill c net framework aspnet cor...
3,1,1,021 assisted development internal application ...
4,2,0,net developer skill c net framework aspnet sql...


In [3]:
from langchain.schema import Document
Document(page_content="represent_text: ...", metadata={"cv_id": ..., "chunk_id": ...})

Document(metadata={'cv_id': Ellipsis, 'chunk_id': Ellipsis}, page_content='represent_text: ...')

In [4]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
  

In [5]:
pip install langchain qdrant-client -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.7/327.7 kB 6.4 MB/s eta 0:00:00:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install transformers accelerate

Note: you may need to restart the kernel to use updated packages.


In [32]:
# # !rm -rf /kaggle/working/*
# import torch
# torch.cuda.empty_cache()

In [ ]:
QDRANT_URL = 'https://bb3f8fb3-5dde-46d6-9961-758fa12ca505.europe-west3-0.gcp.cloud.qdrant.io'
QDRANT_API_KEY = ''
from langchain.embeddings import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

<ipython-input-7-8c74c25d2789>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

### create db

In [ ]:
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

QDRANT_URL = 'https://bb3f8fb3-5dde-46d6-9961-758fa12ca505.europe-west3-0.gcp.cloud.qdrant.io'
QDRANT_API_KEY = ''

documents = [
    Document(
        page_content="represent_text: " + row["text"],
        metadata={"cv_id": row["cv_id"], "chunk_id": row["chunk_id"]}
    )
    for _, row in chunk_df.iterrows()
]

client = QdrantClient(
    url=QDRANT_URL,
    api_key=QDRANT_API_KEY,
)

client.recreate_collection(
    collection_name="cv-nlp",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

qdrant = Qdrant.from_documents(
    documents,
    embedding=embedding_function,
    url=QDRANT_URL,
    prefer_grpc=False,
    collection_name="cv-nlp",
    api_key=QDRANT_API_KEY,
)

print("Số vector đã lưu:", client.count(collection_name="cv-nlp").count)


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer_rerank = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")
model_rerank = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-v2-m3")

def rerank(query: str, docs, top_k=3):

    query = query.strip()
    pairs = [(query, doc.page_content.replace("represent_text: ", "")) for doc in docs]

    inputs = tokenizer_rerank.batch_encode_plus(
        pairs,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    )

    with torch.no_grad():
        scores = model_rerank(**inputs).logits.squeeze()

    ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)

    return [doc for doc, score in ranked[:top_k]]


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [12]:
!pip install bm25s[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.7/646.7 kB 36.5 MB/s eta 0:00:00
  Created wheel for pytrec-eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308216 sha256=4115b188a1e93eab69b97d597b2e251015903ea0c91f4edd67a70bc1720e1716
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built pytrec-eval


In [19]:
import bm25s

documents = chunk_df['text'].tolist()
indexed_documents = list(enumerate(documents))  # List of (index, text)
corpus_only = [text for _, text in indexed_documents]
tokenized_corpus = bm25s.tokenize(corpus_only)
bm25_retriever = bm25s.BM25(corpus=corpus_only)
bm25_retriever.index(tokenized_corpus)

Split strings:   0%|          | 0/17281 [00:00<?, ?it/s]

BM25S Count Tokens:   0%|          | 0/17281 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/17281 [00:00<?, ?it/s]

In [14]:
def rrf_fusion(bm25_results, embedding_results, k=10, rrf_k=60):
    scores = {}
    for rank, idx in enumerate(bm25_results):
        scores[idx] = scores.get(idx, 0) + 1 / (rrf_k + rank + 1)
    for rank, doc in enumerate(embedding_results):
        idx = int(doc.metadata["index"])
        scores[idx] = scores.get(idx, 0) + 1 / (rrf_k + rank + 1)
    
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [idx for idx, _ in ranked[:k]]


In [24]:
def search_cv_with_rrf_rerank(query: str, top_k=10, rerank_top_k=5):
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.vectorstores import Qdrant
    from qdrant_client import QdrantClient
    from langchain.schema import Document
    from collections import defaultdict

    # Embedding-based retrieval
    query_emb_input = "represent_text: " + query
    client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY, prefer_grpc=False)
    qdrant_store = Qdrant(client=client, collection_name='cv-nlp', embeddings=embedding_function)
    emb_results = qdrant_store.similarity_search(query_emb_input, k=top_k)  # retrieve more for fusion

    # Add index info to embedding results
    for doc in emb_results:
        doc.metadata["index"] = chunk_df.index[
            (chunk_df["cv_id"] == doc.metadata["cv_id"]) &
            (chunk_df["chunk_id"] == doc.metadata["chunk_id"])
        ].tolist()[0]

    # BM25 retrieval
    query_tokens = bm25s.tokenize([query])[0]
    bm25_docs, _ = bm25_retriever.retrieve(query_tokens, k=top_k)
    bm25_top_indices = [i for i, text in indexed_documents if text in bm25_docs]

    # Fusion
    fused_indices = rrf_fusion(bm25_top_indices, emb_results, k=top_k)

    # Build Documents for reranking
    fused_docs = []
    for idx in fused_indices:
        row = chunk_df.iloc[idx]
        text = row['text']
        meta = chunk_id_map[idx]
        meta["index"] = idx
        fused_docs.append(Document(page_content="represent_text: " + text, metadata=meta))

    # Rerank
    reranked = rerank(query, fused_docs, top_k=rerank_top_k)

    # Group and merge
    grouped = defaultdict(list)
    for doc in reranked:
        cv_id = doc.metadata.get("cv_id")
        chunk_id = doc.metadata.get("chunk_id", -1)
        grouped[cv_id].append((chunk_id, doc.page_content.replace("represent_text: ", "")))

    merged_results = []
    for cv_id, chunks in grouped.items():
        chunks = sorted(chunks, key=lambda x: x[0])
        full_text = "\n".join(chunk for _, chunk in chunks)
        print(f"\nCV ID: {cv_id}")
        print(full_text)


In [9]:
def search_cv_with_rerank_and_merge(query: str, top_k=5, rerank_top_k=3):
    from langchain.embeddings import HuggingFaceEmbeddings
    from langchain.vectorstores import Qdrant
    from qdrant_client import QdrantClient
    from collections import defaultdict

    query_emb_input = "represent_text: " + query

    client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY, prefer_grpc=False)
    qdrant_store =  Qdrant(client=client, collection_name='cv-nlp', embeddings=embedding_function)

    results = qdrant_store.similarity_search(query_emb_input, k=top_k)
    if not results:
        print("Không tìm thấy kết quả.")
        return []

    reranked = rerank(query, results, top_k=rerank_top_k)

    grouped = defaultdict(list)
    for doc in reranked:
        cv_id = doc.metadata.get("cv_id")
        chunk_id = doc.metadata.get("chunk_id", -1)
        grouped[cv_id].append((chunk_id, doc.page_content.replace("represent_text: ", "")))

    merged_results = []
    for cv_id, chunks in grouped.items():
        chunks = sorted(chunks, key=lambda x: x[0])  
        full_text = "\n".join(chunk for _, chunk in chunks)
        print(f"\nCV ID: {cv_id}")
        print(full_text)


In [25]:
search_cv_with_rerank_and_merge("We are seeking a highly skilled and motivated software engineer with experience in both front-end and back-end development. The ideal candidate will have extensive hands-on experience working with React for building dynamic and responsive user interfaces, as well as expertise in Node.js for developing scalable and high-performance back-end systems.")


CV ID: 1000
back end developer engineer skill java nodejs microservices sql cloud computing api development docker kubernetes james smith austin tx himalayasappjamessmith highly skilled staff backend engineer year experience designing implementing robust backend system hightraffic application proven track record enhancing system performance reliability innovative solution team leadership professional experience tech innovation inc austin tx staff backend engineer feb present architected implemented microservicesbased architecture improved system scalability optimized database query leading reduction response time api endpoint led team engineer adoption agile methodology resulting increase overall team productivity web solution ltd san francisco ca backend engineer may jan designed developed restful apis served million monthly active user implemented caching strategy decreased server load improved application speed collaborated frontend team optimize application performance resulting i

In [26]:
search_cv_with_rrf_rerank("We are seeking a highly skilled and motivated software engineer with experience in both front-end and back-end development. The ideal candidate will have extensive hands-on experience working with React for building dynamic and responsive user interfaces, as well as expertise in Node.js for developing scalable and high-performance back-end systems.")

Split strings:   0%|          | 0/1 [00:00<?, ?it/s]

BM25S Retrieve:   0%|          | 0/1 [00:00<?, ?it/s]


CV ID: 1000
back end developer engineer skill java nodejs microservices sql cloud computing api development docker kubernetes james smith austin tx himalayasappjamessmith highly skilled staff backend engineer year experience designing implementing robust backend system hightraffic application proven track record enhancing system performance reliability innovative solution team leadership professional experience tech innovation inc austin tx staff backend engineer feb present architected implemented microservicesbased architecture improved system scalability optimized database query leading reduction response time api endpoint led team engineer adoption agile methodology resulting increase overall team productivity web solution ltd san francisco ca backend engineer may jan designed developed restful apis served million monthly active user implemented caching strategy decreased server load improved application speed collaborated frontend team optimize application performance resulting i